In [1]:
# Homework 5 - Samuel Dobrossy and Lothaire Lemarquis

# Necessary tools

import pandas as pd
import csv
import tqdm
import py2neo
from py2neo import Graph, Node, Relationship
import itertools


6549it [00:55, 118.22it/s]


(No data)

In [ ]:
# Import the data

FR_target = pd.read_csv(r"C:\FR_target.csv")

FR_target.drop_duplicates('new_id', inplace = True)

graph = py2neo.Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"),name="neo4j")
graph.delete_all()

try:
    graph.run("CREATE CONSTRAINT ON (n:Streamer) ASSERT n.id_ IS UNIQUE")
except:
    print("Already existing constraints")

transaction_list = []
iteration = 0

for row in tqdm.tqdm(FR_target.iterrows()):
    id_ = row[1]["id"]
    mature = row[1]["mature"]
    new_id = row[1]["new_id"]
    partner = row[1]["partner"]
    views = row[1]["views"]
    days = row[1]["days"]
    transaction_list.append({"id_":id_,"views":views,"new_id":new_id,"days":days,"mature":mature,"partner":partner})
    iteration += 1
    transaction = "UNWIND $json as data CREATE (n:Streamer) SET n = data"
    graph.run(transaction, json=transaction_list)
    transaction_list = []
    
    
graph.run(transaction, json=transaction_list)

In [2]:
print(FR_target)

             id  days  mature  views  partner  new_id
0     150417538   577    True    775    False    4867
1     125723704   861    True   2585    False    3692
2     155971814   523   False   1566    False    3816
3      35832890  2234   False   9713    False     416
4      46787750  1896    True  44529    False    4502
...         ...   ...     ...    ...      ...     ...
6546   29196284  2061   False   3822    False    6546
6547  124327806   880    True  30313    False    2392
6548  144229380   640    True   5022    False     391
6549  131423064   736   False   2658    False    2793
6550   45419243  1929    True   3743    False    1092

[6549 rows x 6 columns]


In [3]:
FR_edges = pd.read_csv(r"C:\FR_edges.csv")
transaction_list = []
transaction = """UNWIND $json as data
    MATCH (a:Streamer),(b:Streamer)
    WHERE a.new_id = data.from AND b.new_id = data.to
    MERGE (a)-[r:RELATION]->(b)
    ON CREATE
    SET r.n_collab = 1
    ON MATCH
    SET r.n_collab = r.n_collab + 1
    """
it = 0
for edge in tqdm.tqdm(FR_edges.groupby(["from"])):
    related_ids = edge[1]["to"]
    for combi in list(itertools.combinations(related_ids, 2)):
        sh1 = combi[0]
        sh2 = combi[1]
        transaction_list.append({"from":sh1,"to":sh2})
        it +=1
        graph.run(transaction, json=transaction_list)
        transaction_list = []
graph.run(transaction, json=transaction_list)

  1%|          | 38/5882 [03:14<8:17:45,  5.11s/it] 


KeyboardInterrupt: 

In [23]:
# Les IDs des Top 10 premiers streamers - avec au moins une collaboration

query = """MATCH (s:Streamer)
WHERE exists(s.id_)
RETURN s
ORDER BY s.views DESC
LIMIT 10"""
data = graph.run(query).to_data_frame()
list(data["s"])

[Node('Streamer', days=1479, id_=71852533, mature=False, new_id=3459, partner=True, views=108432615),
 Node('Streamer', days=1311, id_=84934385, mature=False, new_id=6420, partner=True, views=62713570),
 Node('Streamer', days=2613, id_=24147592, mature=False, new_id=5511, partner=True, views=44534929),
 Node('Streamer', days=2027, id_=41719107, mature=True, new_id=2161, partner=True, views=44374644),
 Node('Streamer', days=386, id_=174955366, mature=False, new_id=2799, partner=True, views=43059742),
 Node('Streamer', days=610, id_=147337432, mature=False, new_id=5968, partner=True, views=42801507),
 Node('Streamer', days=2075, id_=40063341, mature=False, new_id=1751, partner=True, views=38404078),
 Node('Streamer', days=660, id_=142506321, mature=False, new_id=5804, partner=True, views=37754841),
 Node('Streamer', days=2384, id_=29478753, mature=False, new_id=1552, partner=True, views=34295755),
 Node('Streamer', days=2460, id_=27528066, mature=False, new_id=5844, partner=True, views=2

In [ ]:
# Densité de degrés de nœuds. Loi d'échelle de puissance ? Signification ?

import seaborn as sns
query = """
MATCH (p)
WITH p, size((p)-[:RELATION]->()) as degree
RETURN degree;
"""
data = graph.run(query).to_data_frame()
ax = sns.distplot(data["degree"],hist=False)
ax.title.set_text("Density of degrees")


In [ ]:
# Streamers avec la mention "mature" (violence, language grossier, etc..)

query = """
MATCH (s:Streamer)
WHERE s.mature=true
with count(s) as nombre
RETURN nombre
"""
graph.run(query)

# We count 2414 streamers tagged as "mature"

In [ ]:
# Is there a correlation between the "mature" tag and collaboration between streamers ?

query = """
MATCH (p:Streamer)-[r:RELATION]->()
where p.mature = TRUE
RETURN p.mature
"""
graph.run(query)

# VRAI : Theres does semm to be some link between collaboration between streamers and if there content is tagged as "mature"

In [ ]:
# Es ce que les partenaires collaborent plus que les non-partenaires ?

#Number of collaborating partners 

query = """
MATCH (s:Streamer)-[r:RELATION]->()
WHERE s.partner = True
with count(s) as nombre
return nombre
"""
graph.run(query)

## There are 5648 partners that collaborate

In [ ]:
# Partners that don't collaborate

query1 = """
MATCH (s:Streamer)-[r:RELATION]->()
WHERE s.partner = false
with count(s) as nombre
return nombre
"""
graph.run(query1)

#There are more non-collaborating partners than collaborating partners